#   Check the raedme file for more information 


## pdf_structuration_readme.md

* you need to read the readme file to understand how to use the code and the rolle of each file

* you can use the lighning ia to run all the content in the notebook you can create a simple account 

* and the free version is enough lighning.ia

In [ ]:
!pip install PyPDF2 pdfplumber spacy matplotlib pillow
!python -m spacy download fr_core_news_sm

In [ ]:
'''
Automation script for processing large PDFs:
- PDF splitting (each page = one PDF file)
- Extraction of raw text, tables, images
- Automatic description and summary of tables
- Generation of a .txt file (raw text) and .md file (structured with titles, lists, images)
- Use of PyPDF2, pdfplumber, SpaCy, and other common utilities

# Prerequisites and Installation:
pip install PyPDF2 pdfplumber spacy matplotlib pillow
python -m spacy download fr_core_news_sm

# Place your PDF in the same folder as this script.
'''


import os
import shutil
import pdfplumber
from PyPDF2 import PdfReader, PdfWriter
import spacy
from PIL import Image
import matplotlib.pyplot as plt




# If you don't use the lightning ai you should change the path to your local path

In [ ]:



# Dossier de sortie pour la division/le contenu
OUTPUT_DIR = "/teamspace/studios/this_studio/OUTPUT_DIR"
os.makedirs(OUTPUT_DIR, exist_ok=True)
IMAGES_DIR = "extracted_images"
os.makedirs(IMAGES_DIR, exist_ok=True) 
TEXT_DIR="/teamspace/studios/this_studio/extracted_text"
os.makedirs(TEXT_DIR, exist_ok=True)
README_DIR="/teamspace/studios/this_studio/extracted_readme"
os.makedirs(README_DIR, exist_ok=True)


In [19]:

def split_pdf(input_pdf_path):
    pdf_reader = PdfReader(input_pdf_path)
    base = os.path.splitext(os.path.basename(input_pdf_path))[0]
    output_files = []
    for i, page in enumerate(pdf_reader.pages):
        writer = PdfWriter()
        writer.add_page(page)
        file_path = os.path.join(OUTPUT_DIR, f"{base}_page_{i+1}.pdf")
        with open(file_path, "wb") as f_out:
            writer.write(f_out)
        output_files.append(file_path)
    return output_files

In [20]:
def summarize_table(table):
    """Résumé simple : nbre de lignes, colonnes, premiers éléments"""
    if not table or not any(table):
        return "Tableau vide"
    num_rows = len(table)
    num_cols = len(table[0])
    head = table[0]
    description = f"Tableau de {num_rows} lignes et {num_cols} colonnes. En-tête : {head}."
    return description


In [21]:
def extract_image_caption(page, bbox, nlp):
    # Extraire un rectangle sous l'image pour trouver un texte d'accompagnement
    # (Heuristique : boîte juste sous l'image, 30 points de hauteur)
    cap_top = bbox[3]
    cap_bot = cap_top + 30
    try:
        crop = page.within_bbox((bbox[0], cap_top, bbox[2], cap_bot))
        text = crop.extract_text() if crop else ""
        if not text:
            return "Image extraite (pas de description détectée)"
        doc = nlp(text)
        relevant = " ".join([sent.text for sent in doc.sents])
        return relevant or "Image extraite"
    except Exception:
        return "Image extraite"


In [28]:
def extract_content(pdf_path, page_num, nlp):
    result = {
        "text": "",
        "tables": [],
        "tables_desc": [],
        "images": [],
        "images_desc": []
    }
    with pdfplumber.open(pdf_path) as pdf:
        if len(pdf.pages) < 1:
            return result
        page = pdf.pages[0]
        result["text"] = page.extract_text() or ""
        tables = page.extract_tables()
        for tab in tables:
            result["tables"].append(tab)
            desc = summarize_table(tab)
            result["tables_desc"].append(desc)
        for i, img in enumerate(page.images):
            bbox = (img["x0"], img["top"], img["x1"], img["bottom"])
            page_image = page.to_image(resolution=200)
            pil_img = page_image.original  # PIL.Image object (entire page)

            width_pixels, height_pixels = pil_img.size
            scale = width_pixels / float(page.width)

            # Calcul du bounding box en pixels
            pixel_bbox = tuple(int(coord * scale) for coord in bbox)

            # Sécurité : clamp des valeurs
            left = max(0, pixel_bbox[0])
            upper = max(0, pixel_bbox[1])
            right = min(width_pixels, pixel_bbox[2])
            lower = min(height_pixels, pixel_bbox[3])

            if right > left and lower > upper:
                cropped_image = pil_img.crop((left, upper, right, lower))
                img_filename = os.path.join(IMAGES_DIR, f"page{page_num}_img{i+1}.png")
                cropped_image.save(img_filename)
                result["images"].append(img_filename)
                img_caption = extract_image_caption(page, bbox, nlp)
                result["images_desc"].append(img_caption)
            else:
                print(f"⚠️ Bounding box invalide pour l'image {i+1} sur page {page_num}")

    return result


In [29]:


def save_txt_and_md(base_name, page_number, content):
    """Génère les versions .txt et .md dans des dossiers séparés"""

    txt_filename = os.path.join(TEXT_DIR, f"{base_name}_page_{page_number}.txt")
    md_filename  = os.path.join(README_DIR, f"{base_name}_page_{page_number}.md")

    # 1. Fichier texte brut
    with open(txt_filename, "w", encoding="utf-8") as f:
        f.write(content["text"])
        for i, tab in enumerate(content["tables"]):
            f.write("\n\n----------\nTableau %d\n----------\n" % (i+1))
            for row in tab:
                f.write(" | ".join(str(cell) for cell in row) + "\n")

    # 2. Fichier markdown structuré
    with open(md_filename, "w", encoding="utf-8") as f:
        f.write(f"# Page {page_number}\n\n")
        f.write("## Texte principal\n\n")
        f.write(content["text"] + "\n\n")
        
        for i, (tab, desc) in enumerate(zip(content["tables"], content["tables_desc"])):
            f.write(f"## Tableau {i+1}\n")
            f.write(f"- Description : {desc}\n")
            f.write("\n| " + " | ".join(str(cell) if cell is not None else "" for cell in tab[0]) + " |\n")
            f.write("|" + " --- |" * len(tab[0]) + "\n")
            for row in tab[1:]:
                f.write("| " + " | ".join(str(cell) if cell is not None else "" for cell in row) + " |\n")
            f.write("\n")
        
        for i, (img, desc) in enumerate(zip(content["images"], content["images_desc"])):
            f.write(f"## Image {i+1}\n")
            f.write(f"![Image extraite]({img})\n")
            f.write(f"- Légende automatique : {desc}\n\n")


In [30]:
def process_pdf(pdf_path):
    # SpaCy pour FR (ou changer en EN si besoin)
    nlp = spacy.load("fr_core_news_sm")
    all_pages = split_pdf(pdf_path)
    basename = os.path.splitext(os.path.basename(pdf_path))[0]
    for idx, page_file in enumerate(all_pages, 1):
        print(f"Traitement page {idx}...")
        content = extract_content(page_file, idx, nlp)
        save_txt_and_md(basename, idx, content)

In [31]:
process_pdf("/teamspace/studios/this_studio/new.pdf")

CropBox missing from /Page, defaulting to MediaBox


Traitement page 1...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Traitement page 2...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Traitement page 446...
Traitement page 447...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Traitement page 448...
Traitement page 449...
Traitement page 450...


CropBox missing from /Page, defaulting to MediaBox


In [32]:
import os
import glob
import shutil

# Dossier contenant les images extraites
IMAGES_DIR = "extracted_images"
OUTPUT_DIR = "/teamspace/studios/this_studio/OUTPUT_DIR"
TEXT_DIR="/teamspace/studios/this_studio/extracted_text"
README_DIR="/teamspace/studios/this_studio/extracted_readme"



def clean_generated_files(base_name, total_pages):
    """Supprime les fichiers .txt, .md et les images générées"""
    # Supprimer les fichiers texte et markdown
    
    if os.path.exists(TEXT_DIR):
        files = glob.glob(os.path.join(TEXT_DIR, "*.txt"))
        for img_file in files:
            os.remove(img_file)
            print(f"🗑️ Supprimé : {img_file}")
        
        
        if not os.listdir(TEXT_DIR):
            os.rmdir(TEXT_DIR)
            print(f"📁 Dossier supprimé : {TEXT_DIR}")


    if os.path.exists(README_DIR):
        files = glob.glob(os.path.join(README_DIR, "*.md"))
        for img_file in files:
            os.remove(img_file)
            print(f"🗑️ Supprimé : {img_file}")
        
        
        if not os.listdir(README_DIR):
            os.rmdir(README_DIR)
            print(f"📁 Dossier supprimé : {README_DIR}")



    # Supprimer les images extraites
    if os.path.exists(IMAGES_DIR):
        files = glob.glob(os.path.join(IMAGES_DIR, "*.png"))
        for img_file in files:
            os.remove(img_file)
            print(f"🗑️ Supprimé : {img_file}")
        
      
        if not os.listdir(IMAGES_DIR):
            os.rmdir(IMAGES_DIR)
            print(f"📁 Dossier supprimé : {IMAGES_DIR}")

    # Supprimer les images extraites
    if os.path.exists(OUTPUT_DIR):
        files = glob.glob(os.path.join(OUTPUT_DIR, "*.pdf"))
        for img_file in files:
            os.remove(img_file)
            print(f"🗑️ Supprimé : {img_file}")
            
        files = glob.glob(os.path.join(OUTPUT_DIR, "*.png"))
        for img_file in files:
            os.remove(img_file)
            print(f"🗑️ Supprimé : {img_file}")
        
        if not os.listdir(OUTPUT_DIR):
            os.rmdir(OUTPUT_DIR)
            print(f"📁 Dossier supprimé : {OUTPUT_DIR}")
    
# Exemple : tu as généré 5 pages à partir de "new"
clean_generated_files(base_name="new", total_pages=450)


🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_211.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_315.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_390.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_230.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_288.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_297.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_171.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_260.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_94.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_223.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_312.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_274.txt
🗑️ Supprimé : /teamspace/studios/this_studio/extracted_text/new_page_323.txt
